# Placeholder Example
Any notebooks in this "examples" folder will be executed in the test suite by `tests/test_notebooks.py` and then copied to `docs/source/notebooks` where they can be rendered by `nbsphinx` and included in the documentation.
* To add a notebook to the documentation, included it in the `.. toctree::` in `docs/source/examples.rst`